This scorecard contains multigig-enablement data at the State level, including:
Current XGSPON enablement - from LU QUAL
Current Multi-gig enablement - from LU QUAL (with hardcoded polygon id's)
Next month construction funded XGS-PON LUs - from PowerBI XGS Prep dashboard
Next month construction funded LUs with core network = Multigig

## Please do a fresh import for this report:
##### 1. Construction funding info to the end of the year from PowerBI - do not pull again until 2023
##### 2. Current copy of the summarized salesforce report  (because it's already been aggregated)

## Information needed from the engineers:
##### 1. Speedtest server ready?
##### 2. Out of MG & construction funded, how many LU's require additional 100G speedtest servers to support mg
##### 3. Final manual WC exclusions from mg opportunities list with sub info view

In [205]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

ProgrammingError: Oracle Client library has already been initialized

### Queries

In [206]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

In [208]:
# XGS qualifications count at wire center level

query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct LUID) as xgs_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
            

                                """

query2 = """
            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_POLYGONID
                    ,O.UNI_SERVICETYPE
                    ,O.UNI_STATUS
                    ,O.TECH_TYPE
                    ,O.FSAI
                    ,O.OLT_TYPE
                    ,O.ADDRESS
                    ,O.OLT
                    ,O.UNI_ECD
                    --O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  *
                    
            FROM    QD

                                """

df_ora = pd.read_sql(query, con=conn)
df_ora_details = pd.read_sql(query2, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [209]:
%%time

# count of MG enabled LUs at wire center level

mg_query = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in ('1178832','1178694','1178144',
                                          '1179498','1179547','1179121','1179499'  --Phoenix AZ turned up 9/30/22
                                          ) --add in MG polygon IDs here
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct LUID) as mg_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 78.1 ms
Wall time: 708 ms


In [210]:
# check data
print(mg_lu_ora.head())
print(df_ora.head())

  STATE PRIMARY_WIRE_CENTER_ID  MG_ENABLED_CNT
0    WA               PYLPWA01             342
1    CO               ARVDCOMA             196
2    MN               MPLSMNBE             336
3    AZ               PHNXAZSO             850
4    AZ               PHNXAZNE             212
  STATE PRIMARY_WIRE_CENTER_ID  XGS_ENABLED_CNT
0    CO               CLSPCOMA             3108
1    FL               CPCRFLXA               37
2    FL               CPCRFLXB             4763
3    CO               DNVRCOCP              150
4    CO               GLDNCOMA              341


In [211]:
# merge xgs enablement with mg enablement into one dataframe (mg enablement is a subset of xgs enablement)
luqual_enablement_df = df_ora.merge(mg_lu_ora, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID']).fillna(0)
luqual_enablement_df_save = luqual_enablement_df.copy() #export this as detail

In [212]:
# check data
luqual_enablement_df.head()

,STATE,PRIMARY_WIRE_CENTER_ID,XGS_ENABLED_CNT,MG_ENABLED_CNT
0,CO,CLSPCOMA,3108,0.0
1,FL,CPCRFLXA,37,0.0
2,FL,CPCRFLXB,4763,0.0
3,CO,DNVRCOCP,150,0.0
4,CO,GLDNCOMA,341,0.0


### Merge LU QUAL data with MG Readiness worksheet

In [213]:
wc_mgr_df = pd.read_excel(r'Inputs/081622_OpportunityScore_MultiGig_Readiness.xlsx',sheet_name = 'Datasheet')
wc_mgr_df_save = wc_mgr_df.copy() # save for export in details

In [214]:
wc_mgr_df.columns

Index(['STATE', 'PRIMARY_WIRE_CENTER_ID', 'Opportunity Score', 'CBSA',
       'Core Status', 'Earliest Available For Multi-Gig', 'Cost for Multi-Gig',
       'Aug', 'Sep', 'Oct', 'Nov', 'Dec', '2022 Total', 'Jan', 'Feb', 'Mar',
       'Apr', 'May', 'Jun', 'July', 'Aug.1', 'Sep.1', 'Oct.1', 'Nov.1',
       'Dec.1', '2023 Total', 'Total'],
      dtype='object')

In [215]:
# obtain only relevant columns
# 10/7/22 - added opportunity score to the data pull
wc_mgr_df = wc_mgr_df[['STATE','PRIMARY_WIRE_CENTER_ID','Core Status','Opportunity Score']]

In [216]:
# clean data - rename columns
wc_mgr_df = wc_mgr_df.rename(columns = {'Core Status': 'CORE_STATUS',
                                        'Opportunity Score': 'OPP_SCORE'})

In [217]:
# merge mg readiness data with lu qual xgs enablement for wc master list
print(wc_mgr_df.head())
print(luqual_enablement_df.head())

xgs_wc_master_df = wc_mgr_df.merge(luqual_enablement_df, how = 'outer', on = ['STATE','PRIMARY_WIRE_CENTER_ID'])

  STATE PRIMARY_WIRE_CENTER_ID     CORE_STATUS  OPP_SCORE
0    UT               GTVLUTMA        MultiGIG  18.998338
1    ID               NMPAIDMA  Planning - 120  18.288358
2    ID               BOISIDMA        MultiGIG  17.283587
3    ID               CLWLIDMA   XGS-PON - 180  17.012318
4    IA               DESMIAEA  Planning - 360  16.742619
  STATE PRIMARY_WIRE_CENTER_ID  XGS_ENABLED_CNT  MG_ENABLED_CNT
0    CO               CLSPCOMA             3108             0.0
1    FL               CPCRFLXA               37             0.0
2    FL               CPCRFLXB             4763             0.0
3    CO               DNVRCOCP              150             0.0
4    CO               GLDNCOMA              341             0.0


In [218]:
# set core status of wire centers with actual MG enablement to MultiGIG, regardless of worksheet status
xgs_wc_master_df.loc[xgs_wc_master_df['MG_ENABLED_CNT']>0,'CORE_STATUS'] = 'MultiGIG'

In [219]:
# base df containing xgs enablement counts, mg enablement counts, and the core network status at wire center level
print(xgs_wc_master_df.shape) # this should pull more wire centers that are not on mg readiness worksheet but in lu_qual_o
xgs_wc_master_df_save = xgs_wc_master_df.copy() #save a copy for export

(216, 6)


### Getting state-level construction funding view

In [220]:
# xgs_wc_master_df as base table = xgs_wc_master_df

# funding information is also at wire center level, join at wire center level then aggregate up

In [221]:
#Import Funding data - export most recent file from PowerBI
Construction_Funding_df = pd.read_excel(r'FundingDetailsExport/Details_Export_Table.xlsx',sheet_name = 'Export')
Construction_Funding_df_save = Construction_Funding_df.copy() # saving a copy for export later

In [222]:
Construction_Funding_df = Construction_Funding_df[['STATE','PRIMARY_WIRE_CENTER_ID','PHASE_ECD_LOGIC','PHASE_NUM_OF_LUS']]
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].dt.to_period('M')
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].astype(str)

In [223]:
Construction_pivot = Construction_Funding_df.groupby(by = ["STATE","PRIMARY_WIRE_CENTER_ID","PHASE_ECD_LOGIC"], as_index = False).sum()
Construction_pivot = Construction_pivot.pivot(index=['STATE','PRIMARY_WIRE_CENTER_ID'], columns='PHASE_ECD_LOGIC')['PHASE_NUM_OF_LUS'].reset_index()

In [224]:
# check data
Construction_pivot = Construction_pivot.fillna(0)
Construction_pivot_save = Construction_pivot.copy() # saving this for export later
Construction_pivot.head()

PHASE_ECD_LOGIC,STATE,PRIMARY_WIRE_CENTER_ID,2022-09,2022-10,2022-11,2022-12
0,AL,LSVLALXA,0.0,0.0,0.0,142.0
1,AR,DWTTARXA,0.0,0.0,0.0,70.0
2,AR,STTGARXB,0.0,0.0,0.0,40.0
3,AZ,TCSNAZFW,0.0,160.0,248.0,306.0
4,AZ,TCSNAZMA,0.0,1572.0,2517.0,1642.0


In [225]:
# clean data - manually adjust field names
Construction_pivot = Construction_pivot.rename(columns = {'2022-09': '2022SEP_CONSTRUCTION_FUNDED',
                                     '2022-10': '2022OCT_CONSTRUCTION_FUNDED',
                                     '2022-11': '2022NOV_CONSTRUCTION_FUNDED',
                                     '2022-12': '2022DEC_CONSTRUCTION_FUNDED'})

In [226]:
# check data
Construction_pivot.head()

PHASE_ECD_LOGIC,STATE,PRIMARY_WIRE_CENTER_ID,2022SEP_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED
0,AL,LSVLALXA,0.0,0.0,0.0,142.0
1,AR,DWTTARXA,0.0,0.0,0.0,70.0
2,AR,STTGARXB,0.0,0.0,0.0,40.0
3,AZ,TCSNAZFW,0.0,160.0,248.0,306.0
4,AZ,TCSNAZMA,0.0,1572.0,2517.0,1642.0


In [227]:
# join construction funding to base table
Funded_XGS_WC = xgs_wc_master_df.merge(Construction_pivot, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID'])

#### View prep

In [228]:
# Creating view

Funded_State_view = Funded_XGS_WC.drop(['OPP_SCORE'], axis = 1).groupby(['STATE'], as_index = False).sum()

# add grand total row - adjust names manually if needed
Funded_State_view.loc[len(Funded_State_view.index)] = ['Grand Total',
                                                       Funded_State_view['XGS_ENABLED_CNT'].sum(),
                                                       Funded_State_view['MG_ENABLED_CNT'].sum(),
                                                       Funded_State_view['2022SEP_CONSTRUCTION_FUNDED'].sum(),
                                                       Funded_State_view['2022OCT_CONSTRUCTION_FUNDED'].sum(),
                                                       Funded_State_view['2022NOV_CONSTRUCTION_FUNDED'].sum(),
                                                       Funded_State_view['2022DEC_CONSTRUCTION_FUNDED'].sum()]

In [229]:
# check data
Funded_State_view.head()

,STATE,XGS_ENABLED_CNT,MG_ENABLED_CNT,2022SEP_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED
0,AZ,11271.0,1062.0,0.0,2854.0,3578.0,2178.0
1,CO,5304.0,196.0,3174.0,8724.0,2982.0,6468.0
2,FL,13633.0,0.0,2359.0,13032.0,1220.0,12700.0
3,IA,0.0,0.0,0.0,0.0,0.0,248.0
4,ID,0.0,0.0,0.0,0.0,0.0,0.0


In [230]:
# view ready for export 

# adjust names manually if needed
Funded_State_view = Funded_State_view[['STATE','2022SEP_CONSTRUCTION_FUNDED','2022OCT_CONSTRUCTION_FUNDED','2022NOV_CONSTRUCTION_FUNDED','2022DEC_CONSTRUCTION_FUNDED']]
Funded_State_view

,STATE,2022SEP_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED
0,AZ,0.0,2854.0,3578.0,2178.0
1,CO,3174.0,8724.0,2982.0,6468.0
2,FL,2359.0,13032.0,1220.0,12700.0
3,IA,0.0,0.0,0.0,248.0
4,ID,0.0,0.0,0.0,0.0
5,MN,1561.0,3384.0,645.0,5689.0
6,NE,0.0,0.0,0.0,0.0
7,NV,0.0,83.0,0.0,52.0
8,OR,76.0,0.0,0.0,0.0
9,UT,382.0,1847.0,3353.0,3456.0


### Getting multigig opportunity summary view

In [231]:
# adjust names manually if needed
MG_Opportunity_Funding = Funded_XGS_WC[['STATE','PRIMARY_WIRE_CENTER_ID','XGS_ENABLED_CNT','MG_ENABLED_CNT','2022OCT_CONSTRUCTION_FUNDED','CORE_STATUS','OPP_SCORE']]

# adding new column that will get construction funding only if the core status is MultiGIG
MG_Opportunity_Funding['2022OCT_CONSTRUCTION_FUNDED_MG'] = np.where(MG_Opportunity_Funding['CORE_STATUS'] == 'MultiGIG',MG_Opportunity_Funding['2022OCT_CONSTRUCTION_FUNDED'],0)

C:\Windows\Temp\ipykernel_12616\3786977241.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MG_Opportunity_Funding['2022OCT_CONSTRUCTION_FUNDED_MG'] = np.where(MG_Opportunity_Funding['CORE_STATUS'] == 'MultiGIG',MG_Opportunity_Funding['2022OCT_CONSTRUCTION_FUNDED'],0)


In [232]:
# check data

MG_Opportunity_Funding.head()

,STATE,PRIMARY_WIRE_CENTER_ID,XGS_ENABLED_CNT,MG_ENABLED_CNT,2022OCT_CONSTRUCTION_FUNDED,CORE_STATUS,OPP_SCORE,2022OCT_CONSTRUCTION_FUNDED_MG
0,UT,GTVLUTMA,NaN,NaN,0.0,MultiGIG,18.998338,0.0
1,ID,NMPAIDMA,NaN,NaN,NaN,Planning - 120,18.288358,0.0
2,ID,BOISIDMA,NaN,NaN,NaN,MultiGIG,17.283587,NaN
3,ID,CLWLIDMA,NaN,NaN,NaN,XGS-PON - 180,17.012318,0.0
4,IA,DESMIAEA,NaN,NaN,NaN,Planning - 360,16.742619,0.0


In [233]:
MG_Opportunity_Funding_WC_save = MG_Opportunity_Funding.copy() # save copy for export

#### Prep view

In [234]:
# Creating view
MG_Opportunity_Funding_State_view = MG_Opportunity_Funding.groupby(['STATE'], as_index = False)[['XGS_ENABLED_CNT','MG_ENABLED_CNT','2022OCT_CONSTRUCTION_FUNDED','2022OCT_CONSTRUCTION_FUNDED_MG']].sum()

# add grand total row - manually adjust the field names
MG_Opportunity_Funding_State_view.loc[len(MG_Opportunity_Funding_State_view.index)] = ['Grand Total',
                                                       MG_Opportunity_Funding_State_view['XGS_ENABLED_CNT'].sum(),
                                                       MG_Opportunity_Funding_State_view['MG_ENABLED_CNT'].sum(),
                                                       MG_Opportunity_Funding_State_view['2022OCT_CONSTRUCTION_FUNDED'].sum(),
                                                       MG_Opportunity_Funding_State_view['2022OCT_CONSTRUCTION_FUNDED_MG'].sum()]

In [235]:
# check data

MG_Opportunity_Funding_State_view # view ready for export

,STATE,XGS_ENABLED_CNT,MG_ENABLED_CNT,2022OCT_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED_MG
0,AZ,11271.0,1062.0,2854.0,2694.0
1,CO,5304.0,196.0,8724.0,423.0
2,FL,13633.0,0.0,13032.0,0.0
3,IA,0.0,0.0,0.0,0.0
4,ID,0.0,0.0,0.0,0.0
5,MN,4483.0,336.0,3384.0,1874.0
6,NE,0.0,0.0,0.0,0.0
7,NV,0.0,0.0,83.0,0.0
8,OR,74.0,0.0,0.0,0.0
9,UT,810.0,0.0,1847.0,0.0


### Getting Subs views

Getting current MG opportunity

In [236]:
# create original view but with all MG wire centers from above base
# active subs y/n -- pending subs y/n -- earliest due date for pending subs

# create sub view of above with only wire centers in states that have at least one mg enablement
# graph to illustrate MG opportunity

In [237]:
# start with same base - check data
xgs_wc_master_df.head()

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT
0,UT,GTVLUTMA,MultiGIG,18.998338,NaN,NaN
1,ID,NMPAIDMA,Planning - 120,18.288358,NaN,NaN
2,ID,BOISIDMA,MultiGIG,17.283587,NaN,NaN
3,ID,CLWLIDMA,XGS-PON - 180,17.012318,NaN,NaN
4,IA,DESMIAEA,Planning - 360,16.742619,NaN,NaN


In [238]:
# fillna

xgs_wc_master_df[['XGS_ENABLED_CNT','MG_ENABLED_CNT']] = xgs_wc_master_df[['XGS_ENABLED_CNT','MG_ENABLED_CNT']].fillna(0)

In [239]:
# derive MG untapped column
xgs_wc_master_df['MG_UNTAPPED'] = xgs_wc_master_df['XGS_ENABLED_CNT'] - xgs_wc_master_df['MG_ENABLED_CNT']

In [240]:
# check data
xgs_wc_master_df.head()

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED
0,UT,GTVLUTMA,MultiGIG,18.998338,0.0,0.0,0.0
1,ID,NMPAIDMA,Planning - 120,18.288358,0.0,0.0,0.0
2,ID,BOISIDMA,MultiGIG,17.283587,0.0,0.0,0.0
3,ID,CLWLIDMA,XGS-PON - 180,17.012318,0.0,0.0,0.0
4,IA,DESMIAEA,Planning - 360,16.742619,0.0,0.0,0.0


In [241]:
# import salesforce summary to get active/pending subs
sf_df_sum = pd.read_excel(r'Salesforce Data/Salesforce_Summary.xlsx',sheet_name = 'Summary')
sf_df_details = pd.read_excel(r'Salesforce Data/Salesforce_Summary.xlsx',sheet_name = 'Details') 

In [242]:
# get relevant columns in salesforce summary df
sf_df_sum = sf_df_sum[['WIRE_CENTER_ID','Total Active Subscribers','Total Pending Activation Subscribers']].rename(columns = {'WIRE_CENTER_ID':'PRIMARY_WIRE_CENTER_ID'})

In [243]:
# wire centers with active subs
sf_df_sum = sf_df_sum[(sf_df_sum['Total Active Subscribers']>0)|(sf_df_sum['Total Pending Activation Subscribers']>0)]

In [244]:
# check data
sf_df_sum

,PRIMARY_WIRE_CENTER_ID,Total Active Subscribers,Total Pending Activation Subscribers
377,PHNXAZNE,1,5
378,PHNXAZSO,14,7
379,SPRSAZEA,13,2
380,SPRSAZMA,13,0
381,TCSNAZCR,0,1
382,TCSNAZFW,0,3
383,TCSNAZMA,6,8
384,TCSNAZRN,0,1
385,ARVDCOMA,16,2
386,CLSPCOMA,0,5


In [245]:
# get earliest pending activation due date

from datetime import timedelta, date
from pandas.tseries.offsets import DateOffset

yesterday_date = date.today() - timedelta(days=90)
future_date = date.today() + timedelta(days=90)

duedate_mask = (sf_df_details['Account Status']=='Pending Activation') & (pd.to_datetime(sf_df_details['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(sf_df_details['Due Date']).dt.date <= future_date)
sf_df_duedates = sf_df_details.loc[duedate_mask,['Wire Center','Due Date']]

In [246]:
# update Due Date field to date only
sf_df_duedates['Due Date'] = pd.to_datetime(sf_df_duedates['Due Date']).dt.date

In [247]:
# get earliest due date by wire center

sf_df_duedates = sf_df_duedates.groupby(['Wire Center'], as_index = False).agg(np.min).rename(columns = {'Wire Center':'PRIMARY_WIRE_CENTER_ID',
                                                                                                         'Due Date': 'Earliest Due Date'})

In [248]:
# join due dates to summary

sf_df_sum = sf_df_sum.merge(sf_df_duedates, on = 'PRIMARY_WIRE_CENTER_ID', how = 'left')

In [249]:
# join salesforce to xgs_wc_master_df

subs_view = xgs_wc_master_df.merge(sf_df_sum, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

#### Prep view

In [250]:
# check data
subs_view.head()

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Total Active Subscribers,Total Pending Activation Subscribers,Earliest Due Date
0,UT,GTVLUTMA,MultiGIG,18.998338,0.0,0.0,0.0,NaN,NaN,NaN
1,ID,NMPAIDMA,Planning - 120,18.288358,0.0,0.0,0.0,NaN,NaN,NaN
2,ID,BOISIDMA,MultiGIG,17.283587,0.0,0.0,0.0,NaN,NaN,NaN
3,ID,CLWLIDMA,XGS-PON - 180,17.012318,0.0,0.0,0.0,NaN,NaN,NaN
4,IA,DESMIAEA,Planning - 360,16.742619,0.0,0.0,0.0,NaN,NaN,NaN


In [251]:
# update subs columns to Y/N

subs_view['Active Subs Y/N'] = np.where(subs_view['Total Active Subscribers'] > 0, 'Y', 'N')
subs_view['Pending Subs Y/N'] = np.where(subs_view['Total Pending Activation Subscribers'] > 0, 'Y', 'N')

In [252]:
subs_view_save = subs_view.copy() # save copy for export

In [253]:
# drop columns
subs_view = subs_view.drop(['Total Active Subscribers','Total Pending Activation Subscribers'], axis = 1)

In [254]:
# reorder columns

subs_view = subs_view[['STATE','PRIMARY_WIRE_CENTER_ID','CORE_STATUS','OPP_SCORE','XGS_ENABLED_CNT','MG_ENABLED_CNT','MG_UNTAPPED','Active Subs Y/N','Pending Subs Y/N','Earliest Due Date']]

In [255]:
# only wire centers with XGS enablement

subs_view = subs_view[subs_view['XGS_ENABLED_CNT'] > 0]

In [256]:
# ready for export

subs_view

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Active Subs Y/N,Pending Subs Y/N,Earliest Due Date
10,UT,MAGNUTNM,MultiGIG,15.255279,216.0,0.0,216.0,N,Y,2022-10-06
13,CO,DNVRCOCP,MultiGIG,14.734274,150.0,0.0,150.0,N,N,NaN
29,CO,DNVRCOMA,MultiGIG,13.735440,437.0,0.0,437.0,Y,Y,2022-10-07
31,MN,MPLSMNGE,MultiGIG,13.691429,677.0,0.0,677.0,N,N,NaN
37,CO,CLSPCOMA,XGS-PON - 180,13.343808,3108.0,0.0,3108.0,N,Y,2022-09-28
39,MN,CRYSMNCR,Planning - 120,13.288129,200.0,0.0,200.0,N,Y,2022-10-10
40,OR,PTLDOR11,Planning - 360,13.273806,74.0,0.0,74.0,N,N,NaN
47,AZ,PHNXAZNE,MultiGIG,12.720800,212.0,212.0,0.0,Y,Y,2022-09-28
56,UT,STGRUTMA,XGS-PON - 120,12.040135,537.0,0.0,537.0,N,Y,2022-10-05
62,WA,STTLWASU,MultiGIG,11.683616,166.0,0.0,166.0,N,N,NaN


### MG only subs view

In [257]:
# start with subs view
mg_subs_view = subs_view.copy()

In [258]:
# getting only Core Network Status MultiGIG
mg_subs_view = mg_subs_view[mg_subs_view['CORE_STATUS'] == 'MultiGIG'].sort_values(by = ['STATE','PRIMARY_WIRE_CENTER_ID'])

In [259]:
# check data
mg_subs_view

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Active Subs Y/N,Pending Subs Y/N,Earliest Due Date
47,AZ,PHNXAZNE,MultiGIG,12.720800,212.0,212.0,0.0,Y,Y,2022-09-28
74,AZ,PHNXAZSO,MultiGIG,11.211576,5796.0,850.0,4946.0,Y,Y,2022-09-13
156,AZ,TCSNAZCR,MultiGIG,8.356103,400.0,0.0,400.0,N,Y,2022-10-06
171,AZ,TCSNAZMA,MultiGIG,7.669243,3993.0,0.0,3993.0,Y,Y,2022-10-04
168,AZ,TCSNAZRN,MultiGIG,7.889057,108.0,0.0,108.0,N,Y,2022-10-05
87,CO,ARVDCOMA,MultiGIG,10.753984,196.0,196.0,0.0,Y,Y,2022-10-07
13,CO,DNVRCOCP,MultiGIG,14.734274,150.0,0.0,150.0,N,N,NaN
111,CO,DNVRCODC,MultiGIG,9.853355,166.0,0.0,166.0,N,N,NaN
29,CO,DNVRCOMA,MultiGIG,13.735440,437.0,0.0,437.0,Y,Y,2022-10-07
147,CO,GLDNCOMA,MultiGIG,8.669051,341.0,0.0,341.0,Y,Y,2022-10-04


In [260]:
# get data with non-zero untapped MG

mg_subs_untapped_view = mg_subs_view[mg_subs_view['MG_UNTAPPED'] > 0]

In [261]:
# ready for export

mg_subs_untapped_view

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Active Subs Y/N,Pending Subs Y/N,Earliest Due Date
74,AZ,PHNXAZSO,MultiGIG,11.211576,5796.0,850.0,4946.0,Y,Y,2022-09-13
156,AZ,TCSNAZCR,MultiGIG,8.356103,400.0,0.0,400.0,N,Y,2022-10-06
171,AZ,TCSNAZMA,MultiGIG,7.669243,3993.0,0.0,3993.0,Y,Y,2022-10-04
168,AZ,TCSNAZRN,MultiGIG,7.889057,108.0,0.0,108.0,N,Y,2022-10-05
13,CO,DNVRCOCP,MultiGIG,14.734274,150.0,0.0,150.0,N,N,NaN
111,CO,DNVRCODC,MultiGIG,9.853355,166.0,0.0,166.0,N,N,NaN
29,CO,DNVRCOMA,MultiGIG,13.735440,437.0,0.0,437.0,Y,Y,2022-10-07
147,CO,GLDNCOMA,MultiGIG,8.669051,341.0,0.0,341.0,Y,Y,2022-10-04
126,MN,MPLSMNBE,MultiGIG,9.398391,2417.0,336.0,2081.0,Y,N,2022-10-06
31,MN,MPLSMNGE,MultiGIG,13.691429,677.0,0.0,677.0,N,N,NaN


### New tab: Internal document - polygon info

In [263]:
# using mg_subs_untapped_view as base, create polygon info sheet
mg_info_view = mg_subs_untapped_view.copy()

In [264]:
# get relevant polygon info at wire center level
polygon_info = df_ora_details[['PRIMARY_WIRE_CENTER_ID','UNI_NDSJOBNUMBER','UNI_POLYGONID']]

In [265]:
# keep unique values
polygon_info = polygon_info.drop_duplicates(ignore_index=True)

In [266]:
polygon_info

,PRIMARY_WIRE_CENTER_ID,UNI_NDSJOBNUMBER,UNI_POLYGONID
0,ARVDCOMA,N.891436,1178832
1,CLSPCOMA,N.937456,1180102
2,CLSPCOMA,N.937464,1180005
3,CLSPCOMA,N.937453,1179979
4,CLSPCOMA,N.937442,1180006
...,...,...,...
132,CPCRFLXB,N.723415,1177748
133,CPCRFLXB,CPCRFLXDOL2 02-02,625756
134,CPCRFLXB,CPCRFLHWOL1 02-06,614270
135,CPCRFLXB,CPCRFLXEOL1 01-01,625663


In [267]:
polygon_info = polygon_info.groupby(['PRIMARY_WIRE_CENTER_ID','UNI_POLYGONID']) \
   .agg({'UNI_NDSJOBNUMBER' : ','.join}) \
   .reset_index() \
   .reindex(columns=polygon_info.columns)

In [268]:
# join polygon info to mg view

mg_info_view = mg_info_view[['STATE','PRIMARY_WIRE_CENTER_ID']].merge(polygon_info, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

In [269]:
# join to salesforce data at polygon id level
# show only salesforce data that has active subscribers and pending subscribers within 3 months in future
duedate_mask = (sf_df_details['Account Status']=='Pending Activation') & (pd.to_datetime(sf_df_details['Due Date']).dt.date >= date.today()) & (pd.to_datetime(sf_df_details['Due Date']).dt.date <= future_date)
sf_df_details_polygon = sf_df_details[(sf_df_details['Account Status'] == 'Active') | duedate_mask]

In [270]:
sf_df_details_polygon = sf_df_details_polygon[['Polygon ID','Account Status']].groupby(['Polygon ID','Account Status'], as_index = False).size().pivot(index = 'Polygon ID', columns = 'Account Status', values = 'size').reset_index().fillna(0)


In [271]:
# update salesforce polygon column to object
sf_df_details_polygon['Polygon ID'] = sf_df_details_polygon['Polygon ID'].astype(str)

In [272]:
mg_info_view = mg_info_view.rename(columns = {'UNI_POLYGONID': 'Polygon ID'})
mg_info_view = mg_info_view.merge(sf_df_details_polygon[['Polygon ID','Active','Pending Activation']], on = 'Polygon ID', how = 'left').fillna(0)

In [273]:
# reorder columns, rename columns
mg_info_view = mg_info_view[['STATE','PRIMARY_WIRE_CENTER_ID','Polygon ID','UNI_NDSJOBNUMBER','Active','Pending Activation']]
mg_info_view = mg_info_view.rename(columns = {'Polygon ID':'POLYGON_ID',
                               'Active': 'ACTIVE_SUBS_CNT',
                               'Pending Activation': 'PENDING_SUBS_CNT'})

mg_info_view.head()

,STATE,PRIMARY_WIRE_CENTER_ID,POLYGON_ID,UNI_NDSJOBNUMBER,ACTIVE_SUBS_CNT,PENDING_SUBS_CNT
0,AZ,PHNXAZSO,1176356,N.560282,0.0,0.0
1,AZ,PHNXAZSO,1178411,N.901867,0.0,0.0
2,AZ,PHNXAZSO,1178437,N.901870,0.0,0.0
3,AZ,PHNXAZSO,1178758,N.901871,0.0,0.0
4,AZ,PHNXAZSO,1178772,N.901878,0.0,0.0


In [274]:
# join in xgs enablement and mg enablement info

# XGS qualifications count at polygon level

polygon_xgs_query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,count(distinct LUID) as xgs_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
            

                                """


polygon_xgs_enablement_df = pd.read_sql(polygon_xgs_query, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [275]:
# mg enablement at polygon level
polygon_multigig_query = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in ('1178832','1178694','1178144',
                                          '1179498','1179547','1179121','1179499'  --Phoenix AZ turned up 9/30/22
                                          ) --add in MG polygon IDs here
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,count(distinct LUID) as mg_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID


"""

polygon_multigig_enablement_df = pd.read_sql(polygon_multigig_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [276]:
# check data
print(polygon_multigig_enablement_df.head())
print(polygon_xgs_enablement_df.head())

  STATE PRIMARY_WIRE_CENTER_ID UNI_POLYGONID  MG_ENABLED_CNT
0    WA               PYLPWA01       1178144             342
1    AZ               PHNXAZSO       1179498             319
2    AZ               PHNXAZNE       1179499             212
3    MN               MPLSMNBE       1178694             336
4    CO               ARVDCOMA       1178832             196
  STATE PRIMARY_WIRE_CENTER_ID UNI_POLYGONID  XGS_ENABLED_CNT
0    CO               CLSPCOMA       1179971              150
1    CO               CLSPCOMA       1180102              271
2    FL               CPCRFLXB       1179775              448
3    CO               GLDNCOMA       1179674              158
4    AZ               PHNXAZSO       1179123              290


In [277]:
polygon_enablement_mg_xgs = polygon_xgs_enablement_df.merge(polygon_multigig_enablement_df, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID','UNI_POLYGONID']).fillna(0)

In [278]:
polygon_enablement_mg_xgs = polygon_enablement_mg_xgs.rename(columns = {'UNI_POLYGONID': 'POLYGON_ID'})

In [279]:
mg_info_view = mg_info_view.merge(polygon_enablement_mg_xgs, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID','POLYGON_ID'])

In [280]:
mg_info_view['MG_UNTAPPED_LU'] = mg_info_view['XGS_ENABLED_CNT'] - mg_info_view['MG_ENABLED_CNT']

In [281]:
# export

import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_Multigig_Enablement_Scorecard_Report.xlsx"

with pd.ExcelWriter(excelfilename) as writer:
    mg_subs_untapped_view.to_excel(writer, sheet_name='MG_Subs_Untapped_Summary', index = False)
    subs_view.to_excel(writer, sheet_name='XGS_Subs_Summary', index = False)
    MG_Opportunity_Funding_State_view.to_excel(writer, sheet_name='MG_Enablement_Tracker', index = False)
    Funded_State_view.to_excel(writer, sheet_name='Construction_Funded_Outlook', index = False)
    mg_info_view.to_excel(writer, sheet_name='Polygon_info', index = False)
 
    
    # raw data
    df_ora_details.to_excel(writer, sheet_name='LUQUALO_Details', index = False)
    Construction_Funding_df_save.to_excel(writer, sheet_name='XGSPrep_Details_PowerBI', index = False)
    wc_mgr_df_save.to_excel(writer, sheet_name='MG_Readiness_details', index = False)
    
